<a href="https://colab.research.google.com/github/arunbalajiii/Ten-Times-Online-Private-Limited/blob/main/internship_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install feedparser SQLAlchemy spaCy


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=34f8a9951fdb8bb3f2d51b36ee365c418ef34f935954aa5e48460ad20722ecd3
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [13]:
python -m spacy download en_core_web_sm

SyntaxError: invalid syntax (<ipython-input-13-e8a31c0c54ec>, line 1)

In [15]:
import feedparser

# List of RSS feeds
rss_feeds = [
    'http://rss.cnn.com/rss/cnn_topstories.rss',
    'http://qz.com/feed',
    'http://feeds.foxnews.com/foxnews/politics',
    'http://feeds.reuters.com/reuters/businessNews',
    'http://feeds.feedburner.com/NewshourWorld',
    'https://feeds.bbci.co.uk/news/world/asia/india/rss.xml'
]

# Parse and print each feed
for feed_url in rss_feeds:
    feed = feedparser.parse(feed_url)
    print(f"Fetching news from: {feed_url}")
    for entry in feed.entries:
        print(f"Title: {entry.title}")
        print(f"Link: {entry.link}")
        print(f"Published: {entry.published}")
        print(f"Content: {entry.summary[:100]}...")  # Only showing part of content
        print('-' * 40)


Fetching news from: http://rss.cnn.com/rss/cnn_topstories.rss
Title: Some on-air claims about Dominion Voting Systems were false, Fox News acknowledges in statement after deal is announced
Link: https://www.cnn.com/business/live-news/fox-news-dominion-trial-04-18-23/index.html
Published: Wed, 19 Apr 2023 12:44:51 GMT


AttributeError: object has no attribute 'summary'

In [16]:
from sqlalchemy import create_engine, Column, String, Text, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Setup SQLite database
engine = create_engine('sqlite:///news.db')
Base = declarative_base()

# Define the NewsArticle model
class NewsArticle(Base):
    __tablename__ = 'news_articles'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    content = Column(Text)
    published_date = Column(String)
    source_url = Column(String)

# Create the database table
Base.metadata.create_all(engine)

# Function to store articles
Session = sessionmaker(bind=engine)
session = Session()

def store_article(title, content, published_date, source_url):
    article = NewsArticle(title=title, content=content, published_date=published_date, source_url=source_url)
    session.add(article)
    session.commit()


<ipython-input-16-43be09962ea2>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [17]:
import spacy

nlp = spacy.load('en_core_web_sm')

def categorize_article(content):
    doc = nlp(content)
    if 'terrorism' in content or 'protest' in content:
        return 'Terrorism/Protest/Political Unrest'
    elif 'earthquake' in content or 'flood' in content:
        return 'Natural Disasters'
    elif 'good news' in content or 'uplifting' in content:
        return 'Positive/Uplifting'
    else:
        return 'Others'


In [18]:
import feedparser
from database_setup import store_article
from article_classifier import categorize_article

# List of RSS feeds
rss_feeds = [
    'http://rss.cnn.com/rss/cnn_topstories.rss',
    'http://qz.com/feed',
    'http://feeds.foxnews.com/foxnews/politics',
    'http://feeds.reuters.com/reuters/businessNews',
    'http://feeds.feedburner.com/NewshourWorld',
    'https://feeds.bbci.co.uk/news/world/asia/india/rss.xml'
]

# Parse and categorize news articles
for feed_url in rss_feeds:
    feed = feedparser.parse(feed_url)
    print(f"Fetching news from: {feed_url}")
    for entry in feed.entries:
        title = entry.title
        content = entry.summary
        published = entry.published
        source_url = entry.link

        # Categorize the article
        category = categorize_article(content)
        print(f"Categorized as: {category}")

        # Store the article
        store_article(title, content, published, source_url)
        print(f"Stored article: {title}\n{'-'*40}")


ModuleNotFoundError: No module named 'database_setup'